In [1]:
import tflearn
import numpy as np
import argparse
import pickle
import string
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
import preprocessor as p
from collections import Counter
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix 
from tensorflow.contrib import learn
from tflearn.data_utils import to_categorical, pad_sequences
from scipy import stats
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
import tensorflow as tf
import os
os.environ['KERAS_BACKEND']='theano'
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model,Sequential
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers, optimizers

Using Theano backend.


In [2]:
#These Functions are for loading the data
def load_data(filename):
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = [] 
    for i in range(len(data)):
        x_text.append(p.tokenize((data[i]['text'])))
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global HASH_REMOVE
    if(dataset=="twitter"):
        HASH_REMOVE = True
        EPOCHS = 10
        BATCH_SIZE = 128
        MAX_FEATURES = 2
        filename = "/home/yogesh/Desktop/Minor/data/twitter_data.pkl"
    return filename

In [3]:
data_1 = "twitter"
data_2 = "wiki"
model_type ="blstm_attention"
vector_type = "sswe"
embed_size = 50
oversampling_rate = 3
max_document_length=None
EMBED_SIZE = 50
EPOCHS = 8
BATCH_SIZE = 128
MAX_FEATURES = 2
NUM_CLASSES = 2
DROPOUT = 0.25
LEARN_RATE = 0.01
#HASH_REMOVE=True

In [4]:
x_text, labels = load_data(get_filename(data_1))
dict1 = {'racism':1,'sexism':1,'none':0} #Transfer learning only two classes
labels = [dict1[b] for b in labels]        
racism = [i for i in range(len(labels)) if labels[i]==2]
sexism = [i for i in range(len(labels)) if labels[i]==1]
x_text = x_text + [x_text[x] for x in racism]*(oversampling_rate-1)+ [x_text[x] for x in sexism]*(oversampling_rate-1)
labels = labels + [2 for i in range(len(racism))]*(oversampling_rate-1) + [1 for i in range(len(sexism))]*(oversampling_rate-1)
print("Counter after oversampling")
from collections import Counter
print(Counter(labels))

Loading data from file: /home/yogesh/Desktop/Minor/data/twitter_data.pkl
Counter after oversampling
Counter({1: 15162, 0: 11036})


In [5]:
#splitting the data into training set and testing set
X_train, X_test, Y_train, Y_test = train_test_split( x_text, labels, random_state=121, test_size=0.10)

In [22]:
X_train[50]

"rt $MENTION$: that kitchen looks like they have re-enacted '2 girls , $NUMBER$ cup'. $HASHTAG$"

In [6]:
post_length = post_length = np.array([len(x.split(" ")) for x in x_text])
if(data_1 != "twitter"):
    max_document_length = int(np.percentile(post_length, 95))
else:
    max_document_length = max(post_length)
print("Document length : " + str(max_document_length))

Document length : 38


In [7]:
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length, MAX_FEATURES)
vocab_processor = vocab_processor.fit(x_text)
trainX = np.array(list(vocab_processor.transform(X_train)))
testX = np.array(list(vocab_processor.transform(X_test)))

vocab_size = len(vocab_processor.vocabulary_)
print("Vocabulary Size: {:d}".format(vocab_size))

vocab = vocab_processor.vocabulary_._mapping
trainY = np.asarray(Y_train)
testY = np.asarray(Y_test)
       
trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)
testX = pad_sequences(testX, maxlen=max_document_length, value=0.)
trainY = to_categorical(trainY, nb_classes=NUM_CLASSES)
testY = to_categorical(testY, nb_classes=NUM_CLASSES)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Vocabulary Size: 8894


In [8]:
print("Running Model: " + model_type + " with word vector initiliazed with " + vector_type + " word vectors.")

#Attention Layer Class for blstm attention model
class AttLayer(Layer):

    def __init__(self, **kwargs):
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='kernel', 
                                      shape=(input_shape[-1],),
                                      initializer='random_normal',
                                      trainable=True)
        super(AttLayer, self).build(input_shape)

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

model = Sequential()
model.add(Embedding(vocab_size, embed_size, input_length=trainX.shape[1]))
model.add(Dropout(0.25))
model.add(Bidirectional(LSTM(embed_size, return_sequences=True)))
model.add(AttLayer())
model.add(Dropout(0.50))
model.add(Dense(NUM_CLASSES, activation='softmax'))
adam = optimizers.Adam(lr=LEARN_RATE, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Running Model: blstm_attention with word vector initiliazed with sswe word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 38, 50)            444700    
_________________________________________________________________
dropout_1 (Dropout)          (None, 38, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 38, 100)           40400     
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 100)               100       
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 485,402
Trainable params: 485,402
Non-traina

In [9]:
print("Word vectors used: " + vector_type)
def get_embedding_weights(filename, sep):
    embed_dict = {}
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(sep)
        embed_dict[row[0]] = row[1:]
    print('Loaded from file: ' + str(filename))
    file.close()
    return embed_dict

def map_embedding_weights(embed, vocab, embed_size):
    vocab_size = len(vocab)
    embeddingWeights = np.zeros((vocab_size , embed_size))
    n = 0
    words_missed = []
    for k, v in vocab.items():
        try:
            embeddingWeights[v] = embed[k]
        except:
            n += 1
            words_missed.append(k)
            pass
    print("%d embedding missed"%n, " of " , vocab_size)
    return embeddingWeights

def get_embeddings_dict(vector_type, emb_dim, data):
    if vector_type == 'sswe':
        emb_dim==50
        sep = '\t'
        vector_file = '/home/yogesh/Desktop/Minor/embedding-results/sswe-u.txt'
    else:
        print ("ERROR: Please specify a correst model or SSWE cannot be loaded with embed size of: " + str(emb_dim))
        return None
    
    embed = get_embedding_weights(vector_file, sep)
    return embed
#TRAINING THE MODEL
model.layers[0].set_weights([map_embedding_weights(get_embeddings_dict(vector_type, embed_size, data_1), vocab, embed_size)])
model.fit(trainX, trainY, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, verbose=1)

Word vectors used: sswe
Loaded from file: /home/yogesh/Desktop/Minor/embedding-results/sswe-u.txt
1159 embedding missed  of  8894
Epoch 1/8
23578/23578 [==============================] - 34s 1ms/step - loss: 0.5874 - acc: 0.6813
Epoch 2/8
23578/23578 [==============================] - 35s 1ms/step - loss: 0.4760 - acc: 0.7673
Epoch 3/8
23578/23578 [==============================] - 36s 2ms/step - loss: 0.3774 - acc: 0.8318
Epoch 4/8
23578/23578 [==============================] - 37s 2ms/step - loss: 0.3102 - acc: 0.8709
Epoch 5/8
23578/23578 [==============================] - 37s 2ms/step - loss: 0.2694 - acc: 0.8894
Epoch 6/8
23578/23578 [==============================] - 37s 2ms/step - loss: 0.2467 - acc: 0.9008
Epoch 7/8
23578/23578 [==============================] - 37s 2ms/step - loss: 0.2239 - acc: 0.9117
Epoch 8/8
23578/23578 [==============================] - 39s 2ms/step - loss: 0.2052 - acc: 0.9194


In [10]:
scores = model.evaluate(testX, testY, verbose=0)

In [11]:
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 91.79%


In [12]:
model.save_weights("bully.h5")
print("Model Successfully Saved To Disk!")

Model Successfully Saved To Disk!


In [13]:
temp = model.predict(testX)
y_pred  = np.argmax(temp, 1)
for i in range(len(y_pred)):
    if y_pred[i]==1:
        print("Racism or Sexism!")
    else :
        print("None!")

Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or 

Racism or Sexism!
None!
Racism or Sexism!
None!
None!
None!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
None!
None!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or 

In [16]:
testX[0]

array([  10,    1,    3,    3,  633, 8074,   80,    2, 2029, 8599, 2401,
        288,   26,    7, 2633,  180,    9,   68,   52,   27,  474,    6,
        105,   83,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [19]:
tweet1='rt $MENTION$: $HASHTAG$ $HASHTAG$ model ? puhlease. did the local thrift shop put on a "fashion" show. and should we be able to see her…'
tokens=p.tokenize(tweet1)
l=[]
l.append(tokens)
arr=np.array(list(vocab_processor.transform(l)))
res=np.argmax(model.predict(arr),1)
print(res)
if res[0]==1:
    print("Sexist or Racist Comment!")
else :
    print("None!")

[1]
Sexist or Racist Comment!
